In [1]:
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import glob

from spectral_analysis_functions import *
# Using OceanDataLab reader 
#from sentinel2_l1c import *
import numpy as np
from S2_read import *
from s2_angs import *

bands=['B04','B08','B03','B02'];  

S2path='/home/ardhuin/Téléchargements/S2A_MSIL1C_20160429T183252_N0201_R027_T11SMS_20160429T184037.SAFE'
boxi1=[4400,5200,3600,4400] # East West South North : this box is covered by another detector
[imgs1,NX1,NY1,nx1,ny1,dx1,dy1,offspec1,phitrig1,thetav1,loncenter1,latcenter1,detector,indexX,indexY]=S2_read(S2path,boxi1,bands)
boxi=[5600,6400,3600,4400] # East West South North 
[imgs,NX,NY,nx,ny,dx,dy,offspec,phitrig,thetav,loncenter,latcenter,detector,indexX,indexY]=S2_read(S2path,boxi,bands)

S2path='/home/ardhuin/Téléchargements/S2A_MSIL1C_20160419T184432_N0201_R027_T11SMS_20160419T184429.SAFE'
boxi1=[1400,2200,600,1400] # East West South North : this box is covered by another detector
[imgs1,NX1,NY1,nx1,ny1,dx1,dy1,offspec1,phitrig1,thetav1,loncenter1,latcenter1,detector,indexX,indexY]=S2_read(S2path,boxi1,bands)

boxi=[2400,3200,600,1400] # East West South North 
[imgs,NX,NY,nx,ny,dx,dy,offspec,phitrig,thetav,loncenter,latcenter,detector,indexX,indexY]=S2_read(S2path,boxi,bands)


Reading XML file for view and sun angles: /home/ardhuin/Téléchargements/S2A_MSIL1C_20160429T183252_N0201_R027_T11SMS_20160429T184037.SAFE/GRANULE/L1C_T11SMS_A004457_20160429T184037/MTD_TL.xml
Reading file for band  B04 : /home/ardhuin/Téléchargements/S2A_MSIL1C_20160429T183252_N0201_R027_T11SMS_20160429T184037.SAFE/GRANULE/L1C_T11SMS_A004457_20160429T184037/IMG_DATA/T11SMS_20160429T183252_B04.jp2
Reading file for band  B08 : /home/ardhuin/Téléchargements/S2A_MSIL1C_20160429T183252_N0201_R027_T11SMS_20160429T184037.SAFE/GRANULE/L1C_T11SMS_A004457_20160429T184037/IMG_DATA/T11SMS_20160429T183252_B08.jp2
Reading file for band  B03 : /home/ardhuin/Téléchargements/S2A_MSIL1C_20160429T183252_N0201_R027_T11SMS_20160429T184037.SAFE/GRANULE/L1C_T11SMS_A004457_20160429T184037/IMG_DATA/T11SMS_20160429T183252_B03.jp2
Reading file for band  B02 : /home/ardhuin/Téléchargements/S2A_MSIL1C_20160429T183252_N0201_R027_T11SMS_20160429T184037.SAFE/GRANULE/L1C_T11SMS_A004457_20160429T184037/IMG_DATA/T11SMS_

RasterioIOError: '/home/ardhuin/Téléchargements/S2A_MSIL1C_20160429T183252_N0201_R027_T11SMS_20160429T184037.SAFE/GRANULE/L1C_T11SMS_A004457_20160429T184037/IMG_DATA/T11SMS_20160429T183252_B02.jp2' not recognized as a supported file format.

In [ ]:
print('lon,lat:',np.degrees(loncenter),np.degrees(latcenter))

nb=np.shape(bands)[0]


B04med1=np.median(imgs1[0,:,:].flatten())
B04med=np.median(imgs[0,:,:].flatten())
B08med1=np.median(imgs1[1,:,:].flatten())
B08med=np.median(imgs[1,:,:].flatten())

mediansb=0.5*(np.median(imgs,axis=(1, 2))+np.median(imgs1,axis=(1, 2)))
print('means:',mediansb)

ratio=B04med/B04med1
G4=(B04med-B04med1)/(tan(np.radians(offspec[0]))-tan(np.radians(offspec1[0])))/mediansb[0]
G8=(B08med-B08med1)/(tan(np.radians(offspec[1]))-tan(np.radians(offspec1[1])))/mediansb[1]
                         
Bd1=(np.cos(np.radians(offspec1[0])))**4*np.cos(np.radians(thetav1[0]))
Bd=(np.cos(np.radians(offspec[0])))**4*np.cos(np.radians(thetav[0]))
mss=(np.tan(np.radians(offspec1[0]))**2-np.tan(np.radians(offspec[0]))**2)/np.log(ratio*Bd/Bd1)

print('Median brighness in B04 and mss:',B04med1,B04med,mss,ratio,Bd/Bd1,offspec[0],offspec1[0])
array1=np.squeeze(detector[1,:,:])
fig, ax = plt.subplots() 
im=ax.pcolor(indexX,indexY,array1)
fig.colorbar(im)
ax.set_title('detector index and FFT box location')
ax.set_xlabel('pixel index')
ax.set_ylabel('pixel index')
ax.axis('equal')
rect = mp.patches.Rectangle((boxi[0], boxi[2]), boxi[1]-boxi[0], boxi[3]-boxi[2], linewidth=2,
                         edgecolor='k', facecolor="none")
rect1 = mp.patches.Rectangle((boxi1[0], boxi1[2]), boxi1[1]-boxi1[0], boxi1[3]-boxi1[2], linewidth=2,
                         edgecolor='r', facecolor="none")
ax.add_patch(rect1)

ax.add_patch(rect)
plt.show()

In [ ]:
# normalizes the image values
img1=imgs[0,:,:]/mediansb[0]
img2=imgs[1,:,:]/mediansb[1]
plt.imshow(np.flipud(np.transpose(img1)),vmin=0.5, vmax=2) #edit your vmin, vmax and cmap if you don't like greyscale colormap
plt.colorbar()
plt.show()
plt.imshow(np.flipud(np.transpose(img2)),vmin=0.5, vmax=2) #edit your vmin, vmax and cmap if you don't like greyscale colormap
plt.colorbar()
plt.show()

In [ ]:
#img2=img1[0:400,0:800]
#(Eta,Eta_all,kx2,ky2,dkx,dky)=FFT2D_one_array(img1,10,10,16,isplot=0)
(Eta,Etb,ang,angstd,coh,crosr,phases,kx2,ky2,dkx,dky)=FFT2D_two_arrays(img1,img2,10,10,8,isplot=0)
print(np.shape(Eta))
print(np.shape(kx2))
print(np.shape(ky2))


phiblind1=phitrig[0]+90.  # blind azimuth for first band
phiblind2=phitrig[1]+90.  # blind azimuth for 3rd band
phiblind3=phitrig[2]+90.  # blind azimuth for 3rd band
kN=50
xkblind=[-1.4*kN*np.sin(np.radians(phiblind2)) , 1.4*kN*np.sin(np.radians(phiblind2))]
ykblind=[-1.4*kN*np.cos(np.radians(phiblind2)) , 1.4*kN*np.cos(np.radians(phiblind2))]


X = np.arange(0,nx*dx,dx) # from 0 to (nx-1)*dx with a dx step
Y = np.arange(0,ny*dy,dy)

fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)
_=plt.gca().set_aspect('equal', adjustable='box')

# warning: we use an array img1 that is written with X in the first (number 0) dimension and Y in the second
#          but pcolormesh uses the opposite order: hence the use of transpose 

im=axs[0].pcolormesh(X/1000,Y/1000,np.transpose(img1),cmap='seismic',norm = mcolors.Normalize(vmin=0.5, vmax=2))
_=plt.colorbar(im,ax=axs[0],label='S2 image band B04')
_=axs[0].set_xlabel('X [km]')
_=axs[0].set_ylabel('Y [km]')
_=axs[0].set_title('Surface')

im=axs[1].pcolormesh(kx2*1000,ky2*1000,10*np.log10((Eta)),norm = mcolors.Normalize(vmin=-30, vmax=20))
_=plt.colorbar(im,ax=axs[1],label='dB')
_=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
_=axs[1].set_xlabel('k_x [cycles / km]')
_=axs[1].set_ylabel('k_y [cycles / km]')
_=axs[1].set_title('Spectrum')
_=plt.tight_layout()

print('variance:',np.var(img1.flatten()), 'sum of spectrum:',np.sum(Eta.flatten())*dkx*dky )


fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)
_=plt.gca().set_aspect('equal', adjustable='box')

im=axs[0].pcolormesh(kx2*1000,ky2*1000,coh,norm = mcolors.Normalize(vmin=0, vmax=1))
_=plt.colorbar(im,ax=axs[0],label=' ')
_=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
_=axs[1].set_xlabel('k_x [cycles / km]')
_=axs[1].set_ylabel('k_y [cycles / km]')
_=axs[0].set_title('coherence B02-B04')

im=axs[1].pcolormesh(kx2*1000,ky2*1000,np.degrees(ang),norm = mcolors.Normalize(vmin=-180, vmax=180))
_=plt.colorbar(im,ax=axs[1],label='degrees')
_=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
_=axs[1].set_xlabel('k_x [cycles / km]')
_=axs[1].set_ylabel('k_y [cycles / km]')
_=axs[1].set_title('cross-spectrum phase')
_=plt.tight_layout()

fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)
_=plt.gca().set_aspect('equal', adjustable='box')

im=axs[0].pcolormesh(kx2*1000,ky2*1000,10*np.log10((crosr)),norm = mcolors.Normalize(vmin=-30, vmax=20))
_=plt.colorbar(im,ax=axs[0],label=' ')
_=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
_=axs[1].set_xlabel('k_x [cycles / km]')
_=axs[1].set_ylabel('k_y [cycles / km]')
_=axs[0].set_title('real part of X-spec')

im=axs[1].pcolormesh(kx2*1000,ky2*1000,10*np.log10((Eta)),norm = mcolors.Normalize(vmin=-30, vmax=20))
_=plt.colorbar(im,ax=axs[1],label='dB')
_=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
_=axs[1].set_xlabel('k_x [cycles / km]')
_=axs[1].set_ylabel('k_y [cycles / km]')
_=axs[1].set_title('Spectrum')
_=plt.tight_layout()


In [ ]:
dir2a=np.degrees(np.arctan2(kx2,ky2))
kn=np.sqrt(kx2**2+ky2**2)*(2*np.pi)+1E-20
kncpk=kn/(2*np.pi)*1000

# Let's now divide by MTF ... 
# 
print('G values:',G4,G8,np.shape(dir2a-phiblind2))
MTF=1/(G4*kn*np.sin(np.radians(dir2a-phiblind2)))**2
J2=np.where((coh < 0.01) | (MTF > 300) )
MTF[J2]=0
Etb=Eta
Etb[J2]=0
Ek=crosr*MTF

fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)
_=plt.gca().set_aspect('equal', adjustable='box')

# warning: we use an array img1 that is written with X in the first (number 0) dimension and Y in the second
#          but pcolormesh uses the opposite order: hence the use of transpose 

im=axs[0].pcolormesh(kx2*1000,ky2*1000,10*np.log10((MTF)),norm = mcolors.Normalize(vmin=-200, vmax=30))
_=plt.colorbar(im,ax=axs[0],label='MTF (dB)')
_=axs[1].set_xlabel('k_x [cycles / km]')
_=axs[1].set_ylabel('k_y [cycles / km]')
_=axs[0].set_title('MTF')

im=axs[1].pcolormesh(kx2*1000,ky2*1000,10*np.log10((Ek)),norm = mcolors.Normalize(vmin=-20, vmax=40))
_=plt.colorbar(im,ax=axs[1],label='dB')
_=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
_=axs[1].set_xlabel('k_x [cycles / km]')
_=axs[1].set_ylabel('k_y [cycles / km]')
_=axs[1].set_title('Spectrum')
_=plt.tight_layout()

varB=np.sum(Etb.flatten())*dkx*dky
sumE=np.sum(Ek.flatten())*dkx*dky


print('sum of spectrum:',sumE,sqrt(sumE), 4*sqrt(sumE), np.sum(Etb.flatten())*dkx*dky  )


In [ ]:
dirC1=110
dirC2=120
dt=-1


J =np.where((coh > 0.01) & (dir2a < dirC2) & (dir2a > dirC1) & (np.degrees(angstd)  < 60) )
J2=np.where((coh > 0.01) & (dir2a < dirC2) & (dir2a > dirC1) )
dispt=np.sqrt(9.81/kn)
Cimg=ang/kn/dt


plt.plot(kn[J]*1000,dispt[J],'kx',linewidth=3,label='Airy theory') 
plt.plot(kn[J]*1000,Cimg[J],'bo',linewidth=1,label='from image X-spectra') 
plt.grid(True)
plt.xlabel('k (cycles / km)')
plt.ylabel('phase speed (m/s)')
plt.legend()